Paula knows a function $f: bool \rightarrow int$.

Valeria wants to query Paula for $f(true)$. The easiest way is for Valerie to simply ask for the value.

In [19]:
class Paula:
    def __init__(self):
        self.valerie = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.paula = None

    def get_f(self):
        value = self.paula.send_f(True)
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula

In [20]:
protocol = Protocol()
value = protocol.valerie.get_f()

Paula recieves parameter True.
Paula sends value 1000.
Valerie obtains value 1000.


Now Valerie doesn't want to tell Paula that she wants the value of $f$ for parameter $True$. She wants this transfer to be oblivious. The easiest way to do this is to ask for both of them, so Paula wouldn't know which value Valerie really wants.

In [21]:
class Paula:
    def __init__(self):
        self.valerie = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.paula = None
        self.parameter = True

    def get_f(self):
        value_for_true = self.paula.send_f(True)
        value_for_false = self.paula.send_f(False)
        
        value = value_for_true if self.parameter else value_for_false
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula

In [23]:
protocol = Protocol()
value = protocol.valerie.get_f()

Paula recieves parameter True.
Paula sends value 1000.
Paula recieves parameter False.
Paula sends value 500.
Valerie obtains value 1000.


This works okay for Valerie, whose privacy is respected, but what if Paula doesn't want to give too much information either?

Notice that in the first naive version Paula only gives one result to Valerie but in the second version she gives her both. This is not desirable.

We'll see a protocol that allows both parties to mantain privacy. That is:
- Paula only reveals one value to Valerie
- Valerie doesn't reveal which of the values she wants

We're going to need that both parties share a RSA key pair to be hide the requests.

TODO: finish the explanation (?)

In [74]:
from Crypto.PublicKey import RSA
import random

def random_noise():
    # N is a really big number
    N = 2**2048
    return random.randrange(N)

class Paula:
    def __init__(self, key, modulus):
        self.valerie = None
        
        self.key = key
        self.modulus = modulus
        self.blinds = {
            True: random_noise(),
            False: random_noise(),
        }
    
    def send_blinds(self):
        print(f"\nPaula sends blinds {self.blinds}.")
        return self.blinds
    
    def calculate_k_b(self, bit, encrypted_key):
        base = (encrypted_key - self.blinds[bit] + self.modulus) % self.modulus
        return pow(base, self.key, self.modulus)
        
    def calculate_v_b(self, k_b, bit):
        return (k_b + self._f(bit)) % self.modulus
    
    def calculate_encrypted_value(self, bit, encrypted_key):
        k_b = self.calculate_k_b(bit, encrypted_key)
        return self.calculate_v_b(k_b, bit)
    
    def send_f(self, encrypted_key):
        print(f"\nPaula recieves parameter {encrypted_key}.")
    
        values = {
            b: self.calculate_encrypted_value(b, encrypted_key) 
            for b in [True, False]
        }
        print(f"\nPaula sends values {values}.")
        return values
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self, key, modulus):
        self.paula = None
        self.parameter = True
        self.key = key
        self.modulus = modulus
        
        self.k = random_noise()

    def get_correct_blind(self):
        return self.paula.send_blinds()[self.parameter]
    
    def get_correct_encrypted_value(self, hidden_key):
        return self.paula.send_f(hidden_key)[self.parameter]
    
    def hide_key(self, blind):
        return (blind + pow(self.k, self.key, self.modulus)) % self.modulus
    
    def retrieve_value(self, encrypted_value,):
        return (encrypted_value - self.k + self.modulus) % self.modulus
    
    def get_f(self):
        blind = self.get_correct_blind()
        print(f"\nValerie calculates correct blind {blind}.")
        
        hidden_key = self.hide_key(blind)
        print(f"\nValerie calculates hidden key {hidden_key}.")
        
        encrypted_value = self.get_correct_encrypted_value(hidden_key)
        print(f"\nValerie receives hidden value {encrypted_value}.")
        
        value = self.retrieve_value(encrypted_value)
        print(f"\nValerie decrypts value {value}.")
    
class Protocol:
    def __init__(self):
        key_pair = RSA.generate(bits=2048)
        paula_key = key_pair.d
        valerie_key = key_pair.e
        modulus = key_pair.n
        
        self.paula = Paula(paula_key, modulus)
        self.valerie = Valerie(valerie_key, modulus)
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula
        
protocol = Protocol()
value = protocol.valerie.get_f()


Paula sends blinds {True: 22978217445266776713287324742515604783172045074119006109647484519941115814358773719886981873100633508167539775433630877738297283644002127070834460104612208274976016644453101940777331690957331436033215201473390583840792408127248411343793170099070642980081569449796535152138123129060595504712138705104884151657908395715618830399861675756865160890995783103347585597547415892787241403920322787971838353563087893170189308309862477641052539900222106607647352104180869245082799288218258078009221413341075128142350708881550408230197207521518613617484267392628774237399660137481791299836390826385532175295912297596566534928249, False: 12235082329355976550003260781782138924375960507610687278902450320708975663525223340561026610713685039928955466239886860052274069026357429320306837522470540563611953106029892972872362040968724805673493394736803575312234698744828724076454096497116702740865970488464561742434633333669964223225488687639279806699090331718617920923980452794154932452070